In [1]:
from reprojection import metashape_utils as mu
from reprojection import reprojection_database as rdb
from reprojection import camera_utils as cu
from reprojection import annotations

from object_detection import sahi_inference as sahi
from object_detection import fifty_one_utils as fou

from ipywidgets import widgets

import Metashape
import pandas as pd
import os


# Pipeline initialisation

Define here your project directory, the path to the metashape project, and the path to the object detection model.

In [ ]:
# Define directories
project_dir = r"D:\metashape\victorHD\project"
project_name = "VictorHD"
metashape_project_path = r"D:\metashape\victorHD\VictorHD.psx"

object_detection_model_path = r"D:\model_training\trained_models\coco_multilabel_yolov11l_datarmor\train_yolo11l_100e_2000imgsz_datarmor\weights\best.pt"

In [2]:
# Create project_dir if it does not exist
if not os.path.exists(project_dir):
    os.makedirs(project_dir)

# Load the metashape project
doc = Metashape.Document()
doc.open(metashape_project_path)

global chunk

def handle_dd_change(change):
    global chunk
    for c in doc.chunks:
        if c.label == change.new:
            chunk = c
            break

if len(doc.chunks) == 1:
    chunk = doc.chunk
else:
    print("More than one chunk found, select one")
    dd = widgets.Dropdown(
        options=[c.label for c in doc.chunks],
        value=doc.chunks[0].label,
        description='Chunk label:',
        disabled=False,
    )
    dd.observe(handle_dd_change, names='value')
    display(dd)



## 1. Get the overlapping images from the model

In [ ]:
export_dir  = os.path.join(project_dir, "overlapping_images")
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

temp_chunk = mu.get_overlapping_images(chunk, export_dir)
doc.remove([temp_chunk])


## 2. Object detection

In [ ]:
dataset = fou.import_image_directory(export_dir, "overlapping_image_dataset")

dataset_inf = sahi.sahi_inference(dataset, object_detection_model_path)

inference_report = fou.fo_to_csv(dataset_inf)
inference_report_path = os.path.join(project_dir, "inference_report.csv")
pd.DataFrame(inference_report).to_csv(inference_report_path, index = False)

## 3. Reprojection

First we create a reprojection database, were all the reprojection will be stored, alongside information on the annotation and the camera. This allows for a fast and easy access to the data.

In [ ]:
db_dir = os.path.join(project_dir, f"reprojection_{project_name}")
if not os.path.exists(db_dir):
    os.makedirs(db_dir)

cameras_reprojectors = cu.chunk_to_camera_reprojector(chunk, db_dir)  ## Get cameras from chunk and create a camera reprojector object for each

create_db = True #  If crash, set to False and run again
if create_db:
    print("Creating DB and adding annotations and reprojections")
    session = annotations.inference_report_to_reprojection_database(db_dir, inference_report_path, cameras_reprojectors)

else:
    print("Getting DB from existing")
    session, path = rdb.open_reprojection_database_session(db_dir, False)


Then, we can filter duplicates and convert the annotations to individuals

In [ ]:
session = annotations.annotations_to_individuals(session, cameras_reprojectors, db_dir)

The annotations are then plotted onto the metashape project

In [ ]:
mu.plot_reprojection_db(session, chunk, project_name, db_dir)

doc.save()
